# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import precision_recall_fscore_support, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
import pickle

import nltk
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql("SELECT * FROM Cleaned_Data", engine)
X = df['message']
Y = df.iloc[:,4:]

In [3]:
df.isnull().sum()

id                            0
message                       0
original                  16046
genre                         0
related                       0
request                       0
offer                         0
aid_related                   0
medical_help                  0
medical_products              0
search_and_rescue             0
security                      0
military                      0
child_alone                   0
water                         0
food                          0
shelter                       0
clothing                      0
money                         0
missing_people                0
refugees                      0
death                         0
other_aid                     0
infrastructure_related        0
transport                     0
buildings                     0
electricity                   0
tools                         0
hospitals                     0
shops                         0
aid_centers                   0
other_in

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
def model_pipeline():
    forest = RandomForestClassifier()
    clf = MultiOutputClassifier(forest)
    pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', clf)
                    ])
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
# split the training set and test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, shuffle = True, random_state=42)
# train pipeline
model = model_pipeline()
model.fit(X_train, Y_train)
# predict the test set
Y_pred = model.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
print(classification_report(Y_test.values, Y_pred, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.81      0.93      0.87      5934
               request       0.82      0.34      0.48      1332
                 offer       0.00      0.00      0.00        36
           aid_related       0.75      0.53      0.62      3219
          medical_help       0.68      0.08      0.15       638
      medical_products       0.61      0.08      0.14       418
     search_and_rescue       0.50      0.04      0.07       192
              security       0.00      0.00      0.00       144
              military       0.36      0.04      0.07       245
           child_alone       0.00      0.00      0.00         0
                 water       0.90      0.23      0.36       500
                  food       0.88      0.26      0.40       878
               shelter       0.82      0.21      0.33       705
              clothing       0.60      0.03      0.05       115
                 money       0.80      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
# first let's see what parameters are included in the pipeline
model.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f931cd94048>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [10]:
parameters = {'vect__ngram_range': ((1, 2),),
              'vect__max_df': (0.75, 1.0),
              'vect__max_features': (5000, 10000),
              'clf__estimator__n_estimators': [100],
              'clf__estimator__min_samples_split': [2, 3]}

model = model_pipeline()

cv = GridSearchCV(model, param_grid=parameters, verbose=5)
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, vect__max_df=0.75, vect__max_features=5000, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, vect__max_df=0.75, vect__max_features=5000, vect__ngram_range=(1, 2), score=0.2560078469838156, total= 4.6min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, vect__max_df=0.75, vect__max_features=5000, vect__ngram_range=(1, 2) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  5.0min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, vect__max_df=0.75, vect__max_features=5000, vect__ngram_range=(1, 2), score=0.262873957822462, total= 4.6min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, vect__max_df=0.75, vect__max_features=5000, vect__ngram_range=(1, 2) 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 10.1min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, vect__max_df=0.75, vect__max_features=5000, vect__ngram_range=(1, 2), score=0.26123916952754617, total= 4.6min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, vect__max_df=0.75, vect__max_features=10000, vect__ngram_range=(1, 2) 


...

[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, vect__max_df=0.75, vect__max_features=10000, vect__ngram_range=(1, 2), score=0.25698871996076506, total= 4.9min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, vect__max_df=0.75, vect__max_features=10000, vect__ngram_range=(1, 2) 


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 20.7min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, vect__max_df=0.75, vect__max_features=10000, vect__ngram_range=(1, 2), score=0.2574791564492398, total= 4.9min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, vect__max_df=0.75, vect__max_features=10000, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, vect__max_df=0.75, vect__max_features=10000, vect__ngram_range=(1, 2), score=0.26042177538008826, total= 5.0min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, vect__max_df=1.0, vect__max_features=5000, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, vect__max_df=1.0, vect__max_features=5000, vect__ngram_range=(1, 2), score=0.25420957985940823, total= 4.6min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, vect__max_df=1.0, vect__max_features=5000, vect__ngram_range=(1, 2) 

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 124.2min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__ngram_range': ((1, 2),), 'vect__max_df': (0.75, 1.0), 'vect__max_features': (5000, 10000), 'clf__estimator__n_estimators': [100], 'clf__estimator__min_samples_split': [2, 3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=5)

So based on the score, the combination of "clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, vect__max_df=1.0, vect__max_features=5000, vect__ngram_range=(1, 2)" works best.  
(Note: I have tried many other parameters already, but it took very long to run grid search and has been interrupted a few times. Below is one example.)

In [ ]:
# cell run on 01/05/2021
parameters = {'vect__ngram_range': ((1, 2),),
              'vect__max_df': (0.75, 1.0),
              'vect__max_features': (5000, 10000),
              'tfidf__use_idf': (True, False),
              'clf__estimator__n_estimators': [50, 100],
              'clf__estimator__min_samples_split': [2, 3]}

#parameters_grid = {'classifier__estimator__n_estimators': [10, 20, 40]}
model = model_pipeline()

cv = GridSearchCV(model, param_grid=parameters, verbose=5)
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__max_df=0.75, vect__max_features=5000, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__max_df=0.75, vect__max_features=5000, vect__ngram_range=(1, 2), score=0.25976786006212194, total= 2.6min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__max_df=0.75, vect__max_features=5000, vect__ngram_range=(1, 2) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.9min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__max_df=0.75, vect__max_features=5000, vect__ngram_range=(1, 2), score=0.26058525420957984, total= 2.6min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__max_df=0.75, vect__max_features=5000, vect__ngram_range=(1, 2) 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  5.7min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__max_df=0.75, vect__max_features=5000, vect__ngram_range=(1, 2), score=0.26140264835703775, total= 2.6min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__max_df=0.75, vect__max_features=10000, vect__ngram_range=(1, 2) 


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  8.6min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__max_df=0.75, vect__max_features=10000, vect__ngram_range=(1, 2), score=0.25976786006212194, total= 2.8min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__max_df=0.75, vect__max_features=10000, vect__ngram_range=(1, 2) 


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 11.7min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__max_df=0.75, vect__max_features=10000, vect__ngram_range=(1, 2), score=0.2600948177211051, total= 2.8min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__max_df=0.75, vect__max_features=10000, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__max_df=0.75, vect__max_features=10000, vect__ngram_range=(1, 2), score=0.2564982834722903, total= 2.8min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=5000, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, tfidf__use_idf=True, vect__max_df=1.0, vect__max_features=5000, vect__ngram_range=(1, 2), score=0.25143043975805135, total= 2.6min
[CV] clf__estimator__min_samples_split=2, clf

In [11]:
Y_pred_2 = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
print(classification_report(Y_test.values, Y_pred_2, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.81      0.96      0.88      5934
               request       0.85      0.49      0.62      1332
                 offer       0.00      0.00      0.00        36
           aid_related       0.75      0.64      0.69      3219
          medical_help       0.72      0.11      0.19       638
      medical_products       0.78      0.17      0.28       418
     search_and_rescue       0.63      0.06      0.11       192
              security       0.00      0.00      0.00       144
              military       0.42      0.06      0.11       245
           child_alone       0.00      0.00      0.00         0
                 water       0.84      0.54      0.66       500
                  food       0.83      0.70      0.76       878
               shelter       0.79      0.48      0.60       705
              clothing       0.73      0.14      0.23       115
                 money       0.58      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


The "recall" score has improved from 0.44 to 0.55 (and therefore f1-score has improved from 0.50 to 0.59). If I have enough time, I would do more grid search and keep improving the model.

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

So here I try to replace RandomForest with SVC:

In [13]:
from sklearn.svm import SVC

In [19]:
def model2_pipeline():
    svc = SVC(verbose=10)
    clf = MultiOutputClassifier(svc)
    pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1,2), max_df=1.0, max_features=5000)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', clf)
                    ])
    return pipeline

There was an error message when I use this new model: because "child_alone" has only one value (all rows are assigned "0" in "child_alone"), SVC does not work. So I dropped the "child_alone" column.

In [28]:
Y_train_temp = Y_train.drop(columns=['child_alone'])

In [31]:
Y_test_temp = Y_test.drop(columns=['child_alone'])

In [33]:
model2 = model2_pipeline()
model2.fit(X_train, Y_train_temp)

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=5000, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip...obability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=10),
           n_jobs=1))])

In [34]:
# predict the test set
Y_pred_model2 = model2.predict(X_test)

In [35]:
print(classification_report(Y_test_temp.values, Y_pred_model2, target_names=Y_test_temp.columns.values))

                        precision    recall  f1-score   support

               related       0.75      1.00      0.86      5934
               request       0.00      0.00      0.00      1332
                 offer       0.00      0.00      0.00        36
           aid_related       0.00      0.00      0.00      3219
          medical_help       0.00      0.00      0.00       638
      medical_products       0.00      0.00      0.00       418
     search_and_rescue       0.00      0.00      0.00       192
              security       0.00      0.00      0.00       144
              military       0.00      0.00      0.00       245
                 water       0.00      0.00      0.00       500
                  food       0.00      0.00      0.00       878
               shelter       0.00      0.00      0.00       705
              clothing       0.00      0.00      0.00       115
                 money       0.00      0.00      0.00       170
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


It seems that SVC does not work very well in this case (or maybe I shall do more grid search and find better parameters).

### 9. Export your model as a pickle file

In [38]:
pickle.dump(cv, open('model.pkl', 'wb'))

Now let's check if the model has been saved successfully, and also use the model to make two simple predictios:

In [39]:
model_load = pickle.load(open('model.pkl', 'rb'))

In [43]:
category_predicted = model_load.predict(['Storm is coming'])[0]
category_predicted

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0])

In [44]:
category_predicted = model_load.predict(['I need water'])[0]
category_predicted

array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

It seems that our model has been successfully saved (and loaded) and made reasonable predictions for the two messages I made.

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.